In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#| echo: false

%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

plt.rcParams.update(**{
    "font.family": "serif",
    "mathtext.fontset": "cm",
    "font.size": 12,

    "axes.spines.right": False,
    "axes.spines.top": False,

    "lines.linewidth": 2.5,
})

::: {.hidden}
$$
 \newcommand{\dd}{{\rm d}}
 \newcommand{\deriv}[2]{\frac{\dd #1}{\dd #2}}
$$
:::

Dans cette leçon, nous verrons comment résoudre numériquement les équations du mouvement issues du principe fondamental de la dynamique.

## Seconde loi de Newton

On considère un corpuscule, ou une particule, de masse $m$, position $\vec{r}$ et vitesse $\vec{v}$ définis au temps $t$. Sous l'action d'une force $\vec{f}$, la quantité de mouvement de la particule $\vec{p} = m\vec{v}$ change selon la seconde loi de Newton :
$$ \deriv{\vec{p}}{t} = \vec{f}, $$
ou, si l'on suppose la masse constante :
$$ m\deriv{\vec{v}}{t} = \vec{f}.$$

La seconde loi de Newton se généralise très bien à un système à $N$ particules. On notera $\{\vec{r}_i\}_{i=1}^N$ l'ensemble des positions, $\{\vec{v}_i\}_{i=1}^N$ l'ensemble des vitesses, $\{m_i\}_{i=1}^N$ l'ensemble des masses et $\{\vec{f}_i\}_{i=1}^N$ l'ensemble des forces totales agissant sur chacune des particules. Nous verrons que le calcul de $\vec{f}_i$ pour chaque particule est central dans les méthodes de simulations discrètes, puisque $\vec{f}_i$ contrôle le mouvement de la particule $i$. On peut alors écrire la seconde loi de Newton pour un système de $N$ particules :


$$ m_i\frac{\mathrm{d}\vec{v}_i}{\mathrm{d}t} = \vec{f}_i.$${#eq-newton}

## Discrétisation

Le problème à trois corps est un problème de physique classique où trois particules interagissent par l'attraction gravitationnelle. Dans le cas général, ce problème n'est pas soluble analytiquement^[Des [solutions particulières](https://en.wikipedia.org/wiki/Three-body_problem) existent, mais le cas général est un parfait exemple de système chaotique], mais on peut le résoudre de manière approchée par des techniques numériques.

Les techniques de résolution numérique sont essentiellement basées sur un principe : celui de transformer une équation différentielle en un ensemble d'équations algébriques (qui ne font pas intervenir la dérivée). C'est le procédé de *discrétisation*. Cependant, ce procédé a un coût, celui de n'obtenir qu'une solution approchée du problème original. L'erreur commise, appelée *erreur de discrétisation*, est souvent inévitable, mais on peut la contrôler dans beaucoup de cas.

On va discuter dans cette leçon de différentes façons d'approximer la solution d'une équation différentielle, et d'une méthode acceptable pour la solution approchée de la seconde loi de Newton.

### Partition du temps

La première étape du processus de discrétisation est de sélectionner des valeurs discrètes de la variable de notre équation différentielle, ici le temps. Dans le problème original $t \in [0, +\infty)$. On défini donc un ensemble de $M$ temps discrets :
$$t_i = \Delta t\cdot i,\quad i \in [0, M-1],$$
ainsi les temps $t_i$ sont uniformément espacés de $\Delta t$, que l'on appellera *pas de temps*. On verra que cette quantité contrôle bon nombre de propriétés numériques de notre système (les propriétés numériques sont uniquement dépendantes des choix de discrétisation, et sont à distinguer des propriétés physiques du système, qui dépendent des équations gouvernant le système).


## Intégration en temps -- premier ordre

Avant de discuter de l'équation de Newton, intéressons-nous à une équation différentielle plus simple :

$$ \dot{y} + y = 0,\quad y(0) = 1, $$

dont on connait la solution, $y(t) = e^{-t}$. Posons l'équation pour les temps discrets $t_i$. On introduit la notation $y_i = y(t_i)$:

$$ \dot{y}_i + y_i = 0,\quad i\in[0, M-1]. $$

### Schéma d'Euler explicite

Pour trouver les valeurs inconnues $y_i$, il nous faut la dérivée de $y$, que l'on ne connait pas. En revanche, on peut l'approximer par différence finie :



$$ \dot{y}_i \approx \frac{y(t_{i+1}) - y(t_i)}{t_{i+1} - t_i} = \frac{y_{i+1} - y_i}{\Delta t} $$

De toute évidence l'approximation devient exacte quand $\Delta t \rightarrow 0$. On peut remplacer notre approximation dans l'équation différentielle :



$$ \frac{y_{i+1} - y_i}{\Delta t} + y_i = 0 \quad \Leftrightarrow \quad y_{i+1} = (1 - \Delta t)y_i,\quad i\in[0, M-1] $$

On a, par approximation de la dérivée en différences finies, transformé une équation différentielle en $M$ équations algébriques, qui définissent en fait une relation de récurrence pour la suite $\{y_i\}_{i=0}^{M-1}$. Puisque la relation de récurrence définit une suite géométrique, on peut écrire $y_i$ directement :

$$ y_i = (1 - \Delta t)^i y_0,$$
on remarque alors une propriété particulière : si $\Delta t < 1$ la limite de la suite est nulle et la suite est monotone, comme la solution analytique, mais si $2 > \Delta t > 1$ la suite oscille (si $\Delta = 1$ la suite est nulle) et si $\Delta t > 2$ la suite diverge. C'est l'exemple parfait d'une propriété numérique, ici que $\Delta t = 1$ est un pas de temps critique qui conditionne la bonne marche de la résolution numérique. C'est une caractéristique du schéma d'**Euler explicite** qui est celui que l'on vient de mettre en place : il est **conditionnellement stable**.

::: {.callout-tip title="Exercice"}

Passons à l'expérience numérique : on implémente le schéma d'Euler explicite et on montre l'effet du pas de temps. Pour cela, on reformule les équations algébriques ci-dessus sous forme d'algorithme :

1. Résolution de l'équation différentielle discrétisée pour trouver la dérivée : $\dot{y}_{i} \gets -y_i$
2. Mise-à-jour de la variable $y_{i+1} \gets y_{i} + \Delta t \dot{y}_i$
3. $i \gets i+1$

Implémentez l'algorithme ci-dessus et essayez de reproduire la @fig-explicit, qui montre le résultat de calculs avec différents pas de temps $\Delta t$. La solution de l'exercice est dans la cellule de code ci-dessous. Commentez les comportements montrées par la @fig-explicit.

:::

In [ ]:
#| label: fig-explicit
#| fig-cap: "Intégration d'Euler explicite avec différents pas de temps"

def euler_explicit(t_final, dt):
    M = int(t_final / dt) + 1
    y = np.zeros(M)
    t = np.arange(M) * dt
    y[0] = 1

    for i in range(M-1):
        y_dot = -y[i]
        y[i+1] = y[i] + dt * y_dot
    return t, y

t = np.linspace(0, 9, 200)
y_exact = np.exp(-t)

cmap = plt.get_cmap('copper')

fig, ax = plt.subplots()

ax.plot(t, y_exact, label="Solution", lw=4, color='k')

for dt in [0.5, 1, 1.5, 2.1]:
    ax.plot(*euler_explicit(t.max(), dt), label=f"$\Delta t = {dt:.1f}$", color=cmap(dt / 2))

ax.legend()
ax.set(xlabel="$t$",
       ylabel="$y(t)$")

plt.show()

### Schéma d'Euler implicite

Au lieu de poser l'équation au temps $t_i$, pour lequel on connaît la valeur de $y_i$ (puisqu'on la calcule à l'itération précédente), posons l'équation au temps $t_{i+1}$ auquel on ne connaît pas encore $y_{i+1}$ (c'est pour cela que le schéma est implicite) :
$$ \dot{y}_{i+1} + y_{i+1} = 0,$$
on doit à présent approximer $\dot y_{i+1}$ par différences finies :
$$ \dot{y}_{i+1} \approx \frac{y(t_{i+1}) - y(t_i)}{t_{i+1} - t_i} = \frac{y_{i+1} - y_i}{\Delta t}, $$
en remplaçant dans l'équation différentielle :
$$ \frac{y_{i+1} - y_i}{\Delta t} + y_{i+1} = 0 \quad \Leftrightarrow \quad y_{i+1} = \frac{1}{1 + \Delta t}y_i,\quad i\in[0, M-1] $$
Au premier abord, on dirait une situation similaire au schéma précédent : on a une relation de récurrence qui définit une suite géométrique. En revanche, on voit que pour tout $\Delta t > 0$, la raison de la suite $1/(1 + \Delta t)$ est plus petite que 1. Le schéma que l'on vient de poser, appelé **Euler implicite** est donc **inconditionnellement stable**.

::: {.callout-tip title="Exercice"}
Faisons la même expérience que précédemment, en formulant les équations sous forme d'algorithme. Comme on a une formulation implicite, l'écriture de l'algorithme est moins évidente. On doit partir de l'équation de différence finie pour $\dot{y}_{i+1}$. En combinant avec l'équation différentielle, on peut écrire que :

\begin{align*}
\dot{y}_{i+1} (= -y_{i+1}) & = \frac{y_{i+1} - y_i}{\Delta t}\\
\Leftrightarrow y_{i+1} & = y_i + \Delta t \dot{y}_{i+1}\\
\Leftrightarrow -\dot{y}_{i+1} &= y_i + \Delta t\dot{y}_{i+1}\\
\Leftrightarrow \dot{y}_{i+1} &= \frac{-1}{1 + \Delta t} y_i
\end{align*}

On peut alors écrire l'agorithme suivant:

1. $\dot{y}_{i+1}\gets \frac{-1}{1 + \Delta t} y_i$
2. $y_{i+1} \gets y_i + \Delta t \dot{y}_{i+1}$
3. $i\gets i + 1$

Implémentez l'algorithme ci-dessus et essayez de reproduire la @fig-implicit, qui montre des calculs par schéma d'Euler implicite pour différents pas de temps. Commentez la différence avec la @fig-explicit.

:::

In [ ]:
#| label: fig-implicit
#| fig-cap: "Intégration d'Euler implicite avec différents pas de temps"

def euler_implicit(t_final, dt):
    M = int(t_final / dt) + 1
    y = np.zeros(M)
    t = np.arange(M) * dt
    y[0] = 1

    for i in range(M-1):
        y_dot = -1 / (1 + dt) * y[i]
        y[i+1] = y[i] + dt * y_dot
    return t, y

t = np.linspace(0, 8, 200)
y_exact = np.exp(-t)

plt.plot(t, y_exact, label="Solution", lw=4, color='k')

for dt in [0.5, 1, 1.5, 2.1]:
    plt.plot(*euler_implicit(t.max(), dt), label=f"$\Delta t = {dt:.1f}$", color=cmap(dt / 2))

plt.legend()
plt.xlabel("$t$")
plt.ylabel("$y(t)$")
plt.show()

On observe, certes, que plus $\Delta t$ est grand, moins la solution numérique est proche de la solution analytique, mais aucune solution numérique n'a de comportement oscillant comme précédemment. On voit également, contrairement à la figure d'Euler explicite, que la pente initiale est fausse pour les solutions approximées.

### Améliorer la convergence

On a vu deux façons de mettre à jour une variable avec la méthode d'Euler :

- Euler explicite, qui utilise $\dot y_{i}$ : $y_{i+1} \gets y_i + \Delta t y_i$
- Euler implicite, qui utilise $\dot y_{i+1}$ : $y_{i+1} \gets y_i + \Delta t y_{i+1}$

Le schéma ci-dessus montre graphiquement ces deux façons de mettre à jour la variable :

In [ ]:
#| label: fig-slopes
#| fig-cap: "Prédictions d'Euler explicite et implicite pour un pas de temps"

fig, ax = plt.subplots()

start = 1
end = 2
dt = 1

t = np.linspace(start - 0.1, end + 0.1, 200)
y = np.exp(-t)

ax.plot(t, y, color='k')
ax.plot([start, end], [np.exp(-start), np.exp(-end)], marker='o', color='r', ls='--')

# Forward euler update
ax.axline((start, np.exp(-start)), slope=-np.exp(-start), color='orange', ls='--', lw=1.5)
ax.plot([end], [np.exp(-start) + dt * (-np.exp(-start))], marker='o', color='r')

# Backward euler update
ax.axline((end, np.exp(-end)), slope=-np.exp(-end), color='gray', ls='--', lw=1.5)
ax.axline((start, np.exp(-start)), slope=-np.exp(-end), color='orange', ls='--', lw=1.5)
ax.plot([end], [np.exp(-start) + dt * (-np.exp(-end))], marker='o', color='r')


ax.set(xlabel='$t$', ylabel="$y(t)$")
plt.show()

On remarque qu'aucune des pentes utilisées pour la mise-à-jour n'est satisfaisante, et que pour atteindre le point rouge sur la courbe noire, la solution analytique, il faut faire un pas avec une pente entre la pente initiale $\dot y_{i}$ et la pente finale $\dot y_{i+1}$. On propose donc le schéma de mise à jour suivant, avec une interpolation linéaire des deux pentes :

$$ y_{n+1} \gets y_i + \Delta t(\beta \dot y_i + (1 - \beta) \dot y_{i+1}),\quad \beta \in [0, 1]$$

Notons que pour $\beta = 1$ on a le schéma explicite et $\beta = 0$ le schéma implicite précédent, mais le schéma est implicite pour toute valeur de $\beta$ non-nulle, car il faut calculer $\dot y_{i+1}$. Résolvons pour l'inconnue $\dot y_{i+1}$, on obtient :

$$ \dot y_{i+1} = \frac{-1}{1 + \Delta t(1 - \beta)}(y_i + \Delta t \beta \dot y_i) $$

On peut alors écrire un algorithme sous la forme d'un schéma de **prédiction-correction** :

1. $\dot y_i \gets -y_i$
2. $y_\mathrm{pred} \gets y_i + \Delta t\beta \dot y_i$ (prédiction)
3. $\dot y_{i+1} \gets \frac{-1}{1 + \Delta t(1 - \beta)}y_\mathrm{pred}$
4. $y_{i+1} \gets y_\mathrm{pred} + \Delta t(1-\beta)\dot y_{i+1}$ (correction)
5. $i \gets i+1$

::: {.callout-important}
La structure "prédiction $\rightarrow$ mise-à-jour de la dérivée $\rightarrow$ correction" est très utilisée par les schémas d'intégration en temps. C'est d'ailleurs cette structure que l'on utilisera pour résoudre l'@eq-newton.
:::

::: {.callout-tip title="Exercice"}
Implémentez l'algorithme ci-dessus, et essayez de reproduire la @fig-comparison, qui compare les trois schémas d'intégrations en temps ($\beta = 0$, $\beta = 1$ et $\beta = 0.5$) pour deux valeurs de $\Delta t$.
:::

In [ ]:
#| label: fig-comparison
#| fig-cap: "Comparaison d'algorithmes d'intégration pour deux pas de temps"

def euler_partial(t_final, dt, beta):
    M = int(t_final / dt) + 1
    y = np.zeros(M)
    t = np.arange(M) * dt
    y[0] = 1

    for i in range(M-1):
        y_dot = -y[i]
        y[i+1] = y[i] + dt * beta * y_dot
        y_dot = -1 / (1 + dt * (1 - beta)) * y[i+1]
        y[i+1] += dt * (1-beta) * y_dot
    return t, y

fig, axs = plt.subplots(1, 2, figsize=(8, 4))

t = np.linspace(0, 8, 200)
y_exact = np.exp(-t)

for ax in axs:
    ax.plot(t, y_exact, label="Solution", lw=4, color='k')

for ax, dt in zip(axs, [1.5, 0.5]):   
    ax.plot(*euler_implicit(t.max(), dt), label="$\\beta = 0$")
    ax.plot(*euler_explicit(t.max(), dt), label="$\\beta = 1$")
    ax.plot(*euler_partial(t.max(), dt, 0.5), label="$\\beta = 0.5$")
    ax.legend()
    ax.set(xlabel='$t$', ylabel='$y(t)$', title=f'$\\Delta t= {dt:.1f}$')

fig.tight_layout()
plt.show()

On remarque que l'approximation avec $\beta = 0.5$ est bien meilleures que les deux schémas précédents. En revanche, cela se fait au prix de deux désavantages :

- L'algorithme est implicite. Ce n'est pas très grave sur un système aussi simple, mais sur de grosses simulations le coût de calcul de $\dot y_{i+1}$ peut être élevé.
- Malgré le fait que l'algorithme soit implicite, il est conditionnellement stable : selon la valeur de $\beta$, $\Delta t$ ne peut pas dépasser une certaine valeur

On a donc gagné en précision en ayant le même coût de calcul que le schéma d'Euler implicite mais sans son avantage principal d'être inconditionnellement stable. C'est à la personne faisant la simulation de choisir l'algorithme adapté !

## Intégration en temps --- second ordre

## Lectures conseillées

- @bonnetAnalyseSolidesDeformables2006, chapitre 9